In [1]:
suppressPackageStartupMessages(library(data.table))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(stringr))
suppressPackageStartupMessages(library(RColorBrewer))

In [2]:
dir = '/nfs/lab/projects/pbmc_snATAC/analysis_v2/run_cicero_paola/'

In [3]:
setwd(dir)

In [4]:
# infile   = "final_conns/coaccessible.matrix"
# outfile  = "final_conns/coaccessible.bedpe"
# annofile = "final_conns/annotated.coaccessible.bedpe"
# B   = '/nfs/lab/publicdata/gencode_v19/gencode.v19.2kb_all_possible_transcripts.merged.bed'
# sed = paste("awk -F \"\\t\" '{print $1,$2}'", infile, "| tail -n +2 | sed 's/_/\\t/g' | pgltools sort >" , outfile) 
# pgl = paste('pgltools intersect1D -a', outfile ,'-b' , B,  '-wa -allA >', annofile)

#system(sed)
#system(pgl)

In [5]:
cells = c(  'b', "mono", 't',"nk")

In [30]:
anno       = read.table(annofile)
anno$loops = paste(anno[,1],anno[,2],anno[,3],anno[,4],anno[,5],anno[,6], sep="_")
anno       = anno[!duplicated(anno),] #remoeve two prom of the same gene
ag         = aggregate(V7~loops, anno, function(x) c("PP", "CP") [((sum( x=='A')/length(x)) %in% c(0,1)) +1] )
coac       = read.table("final_conns/coaccessible.matrix4s", header=T)
totc       = colSums(!is.na(coac[,cells]))
coac$loops = paste(coac$Peak1, coac$Peak2, sep="_")
coac       = merge(coac, ag, by="loops")
coac       = merge(coac, anno[,7:9], by="loops")
colnames(coac)[8:10] = c("type", "prom_side", "genes")

In [148]:
write.table(coac, "final_conns/annotated.coaccessible.matrix4s", sep="\t", quote=F, row.names=F)

In [149]:
stats =  data.frame()
for (ce in cells){
    
    mycel = coac[!is.na(coac[,ce]), c('loops', "type", ce)]
    mycel = mycel[!duplicated(mycel),]
    stats = rbind(stats, table(mycel$type)) 
}

In [150]:
colnames(stats) = c("CP", "PP")
rownames(stats) = cells

In [151]:
stats$tot = totc

In [152]:
stats

,CP,PP,tot
,<int>,<int>,<dbl>
b,62155,25121,158935
mono,70975,36990,254692
t,103296,30551,327048
nk,56467,31713,152213


### filter for cell peaks No

In [141]:
#coacall       = read.table(infile, header=T)

In [142]:
# statsf =  data.frame()
# SUB    = data.frame()
# for (ce in cells){
# fbed = paste0('/nfs/lab/projects/pbmc_snATAC/analysis_v2/rasqual_eur/broad/', ce, '/' , ce, '.filtered.bed')

# fpeaks = read.table(fbed)
# fpeaks = paste(fpeaks[,1], fpeaks[,2]-1, fpeaks[,3], sep="_")
# mycel = coac[!is.na(coac[,ce]), c('loops', "type",'Peak1', 'Peak2', ce)]

# mycel =subset(mycel, Peak1 %in% fpeaks & Peak2 %in% fpeaks)
# mycel = mycel[!duplicated(mycel),c('loops', "type", ce)]

# sub     = subset(coacall, Peak1 %in% fpeaks & Peak2 %in% fpeaks )
# statsf  = rbind(statsf, c(table(mycel$type), sum(!is.na(sub[,ce]))))
    
# sub               = sub[, c('Peak1', 'Peak2', ce)]
# colnames(sub) [3] = 'coacc'
# sub$cell          = ce
# sub               = sub[complete.cases(sub),]
# SUB               = rbind(SUB, sub)
#     }

In [143]:
#colnames(statsf) = colnames(stats)
#rownames(statsf) = rownames(stats)

In [114]:
# widef  = spread(SUB, key=cell, value=coacc)
# widef  = as.data.frame(widef)
# write.table(widef, paste0('final_conns/peak_filtered.coaccessible.matrix'), sep= "\t", quote=F, row.names=F)
# coac = merge(widef, coac[,c('Peak1','Peak2','type','prom_side','genes')], by=c('Peak1','Peak2'))
# write.table(coac, paste0('final_conns/peak_filtered.annotated.coaccessible.matrix'), sep= "\t", quote=F, row.names=F)


In [164]:
pdf("../figures/SummaryCoaccessibleSites.pdf")
stat2  = round(stats/1000,1)  ## unfiltered
#stat3 = statsf/1000  ## filtered for coverage
par(xpd = TRUE, mfcol=c(3,3), las=2, mar = c(4,5,4,4))
bp<-barplot(stat2[,3],  ylab ="# coaccessible sites (x1e3)",names.arg = cells,ylim=c(0,4e+2))
text(bp, stat2[,3],labels=stat2[,3],pos=3)
bp<-barplot(stat2[,1],  ylab ="# coaccessible\n enhancer-promoters(x1e3)",names.arg = cells, ylim=c(0,120))
text(bp, stat2[,1],labels=stat2[,1],pos=3)
bp<-barplot(stat2[,2],  ylab ="# coaccessible\n promoter-promoters(x1e3)",names.arg = cells,ylim=c(0,40))
text(bp, stat2[,2],labels=stat2[,2],pos=3)
# barplot(stat3[,3],  ylab ="# coaccessible sites (x1e3)",names.arg = cells,ylim=c(0,4e+2))
# barplot(stat3[,1],  ylab ="# coaccessible\n enhancer-promoters(x1e3)",names.arg = cells, ylim=c(0,1.2e+2))
# barplot(stat3[,2],  ylab ="# coaccessible\n promoter-promoters(x1e3)",names.arg = cells, ylim = c(0,40))
dev.off()

png 
  2

### Annotate caQTLs

In [6]:
setwd('/nfs/lab/projects/pbmc_snATAC/analysis_v2/')

In [7]:
allqtls = read.table ("summarized_caqtls/run3/EUR_caqtl_leads_run3.tsv", header=T)
allqtls = subset(allqtls, flag_fdr10==TRUE & exclude==FALSE)

In [8]:
clrs = read.table("peaks/color_scheme.tsv")
clrs = rbind(clrs, matrix( c('orange2', 'b','tan4','nk','#7FC97F','mono', 'magenta4', "t"), ncol=2, byrow=T))

colcol  = data.frame(celltypes = clrs$V2, 
                     group= c('mono', 'mono', 'mono', "mono","b", "b", "nk", "nk",
                               't','t','t','t','t','t','mkc','pDC', 'plasma','b','nk','mono','t'))

In [9]:
allqtls$group = colcol$group[match (allqtls$cell, colcol$celltypes  )]

In [10]:
cells = c(  'b', "mono", 't',"nk")

In [11]:
## 1) Annotate all peaks
A = '/nfs/lab/publicdata/gencode_v19/gencode.v19.2kb_all_possible_transcripts.bed'
B = paste0('peaks/pbmc.sorted.merged.bare.bed')
C = paste0('peaks/pbmc.sorted.merged.prom.annot.bed')
#system(paste("bedtools intersect -a", A, "-b", B, "-wa -wb >", C))


pran =read.table(C)
pran$peak = paste0(pran[,5], ":",pran[,6], "-",pran[,7])
prang = aggregate(V4~peak, pran, function(b) paste(unique(b), collapse=","))

In [12]:
coac       = read.table("run_cicero_paola/final_conns/annotated.coaccessible.matrix4s", header=T)

In [13]:
cells

[1] "b"    "mono" "t"    "nk"

In [14]:
ptomoter_qtls  = data.frame()
summary_annot  = data.frame()
genes_enhancer =  data.frame()
for (c in cells) {

## 1) Select caQTL to test
results = subset(allqtls, group==c)
results$peak = gsub( ":|-",  "_", results$Feature)
results = results[!duplicated(results$Feature),]


## 2) Pair with the coaccessible peak (only one qtl)
cic   = coac[!is.na(coac[,c]), c('Peak1','Peak2', c)]

caca = cic[apply(cic[,1:2], 1, function(x)  sum(x %in% results$peak)==2 ),] #### complex coaccessible ca qtls
cic = cic[apply(cic[,1:2], 1, function(x)  sum(x %in% results$peak)==1 ),]
cic$Feature  = unlist(apply(cic[,1:2],  1, function(x)  x [x %in% results$peak]))
cic$coacPeak = unlist(apply(cic[,1:2], 1, function(x)  x [!(x %in% results$peak)]))
cic = cic[!duplicated(cic),]                         

fe = str_split_fixed(cic$Feature,"_",3)
co = str_split_fixed(cic$coacPeak,"_",3)
tab = data.frame(Feature = paste0(fe[,1],":" ,fe[,2],"-",fe[,3]), 
                 coacPeak = paste0(co[,1],":" ,co[,2],"-",co[,3]), coaccess = cic[,c], 
                 dist =abs(as.numeric(fe[,3]) - as.numeric(co[,3])) )


tab$feature_gene = prang$V4[match(tab$Feature, prang$peak)]
tab$coac_gene = prang$V4[match(tab$coacPeak, prang$peak)]
tab$type =c( "EP","P", "PP")  [rowSums(!is.na(tab[,c('feature_gene','coac_gene')])) + !(is.na(tab$feature_gene))]



##remove connsections to the same gene
tab = tab[!duplicated(tab[c("Feature", 'feature_gene','coac_gene','type')]),]

ag2        = aggregate(coac_gene~Feature, tab[tab$type=="EP",], function (x) paste(unique(x), collapse = ","))
ag2$gene_n = sapply( ag2$coac_gene, function(x) str_count(x, ',')+1)
ag2$gene_n[ag2$gene_n>5]<-"6+"
tb2 = table(ag2$gene_n)

## add promoter only
remains = subset(results, !(Feature %in% tab$Feature))
remains$feature_gene = prang$V4[match(remains$Feature, prang$peak)]
remains$type = "P"
remains = subset( remains, !is.na(remains$feature_gene))
tab     = merge(tab, remains[,c('Feature', 'feature_gene', "type")], by=c('Feature', 'feature_gene', "type"), all=T)
tab$group=c

ag = aggregate(type~ Feature,tab, function(b) paste(unique(b), collapse=","))
ag$type[grepl("PP", ag$type )]<-"PP"
tb1 = table(ag$type)
tb1["CC"]<- sum(!duplicated(caca$Peak1, caca$Peak2))  ### complex cases


tb1["NP"] <- sum(!(results$Feature %in% tab$Feature)) - sum(!duplicated(caca$Peak1, caca$Peak2)) ## not promoter
  
               
ptomoter_qtls  = rbind(ptomoter_qtls,tab)
summary_annot  = rbind(summary_annot, tb1)
genes_enhancer = rbind(genes_enhancer, tb2)
               }

In [15]:
colnames(summary_annot)  = paste(c(2,1,3,4,5),names(tb1), sep="_")
colnames(genes_enhancer) = names(tb2)
rownames(summary_annot)  = rownames(genes_enhancer) = cells

In [16]:
summary_annot

,2_EP,1_P,3_PP,4_CC,5_NP
,<int>,<int>,<int>,<int>,<int>
b,339,49,149,46,176
mono,1097,254,361,173,2052
t,1256,200,498,351,486
nk,379,61,179,38,282


In [17]:
genes_enhancer

,1,2,3,4,5,6+
,<int>,<int>,<int>,<int>,<int>,<int>
b,120,74,51,30,14,50
mono,514,250,123,79,33,98
t,431,257,172,116,61,219
nk,132,110,45,34,15,43


In [18]:
genes_enhancer[,1]/rowSums(genes_enhancer)

b      mono         t        nk 
0.3539823 0.4685506 0.3431529 0.3482850

In [19]:
write.table(ptomoter_qtls, "summarized_caqtls/Promoter_associated_caQTLs.tsv", sep="\t", row.names=F, quote=F)

In [20]:
pal = pal = brewer.pal(6,'RdYlBu')[c(1:3,5)]

In [21]:
toplot = t(summary_annot[, c(2,1,3,4)])

In [22]:
toplot

,b,mono,t,nk
1_P,49,254,200,61
2_EP,339,1097,1256,379
3_PP,149,361,498,179
4_CC,46,173,351,38


In [23]:
toplot[2,]/colSums(toplot[1:3,])

b      mono         t        nk 
0.6312849 0.6407710 0.6427840 0.6122779

In [25]:
pdf("figures/caQTL_prom_annotat.pdf")
#layout(matrix(c(1,1,1,1,1,2,2,2,2,2,3,4,5,6,7,8,9,10,11,12), ncol=2, byrow = F))
layout(matrix(c(1,1,1,1,1,    2,3,4,5,  6,6,6,7,7,7,8,8,8      ), ncol=2, byrow = F))
par( las=2, mar = c(4,4,2,8))
barplot(toplot, ylim=c(0,2500), ylab="# caQTL-peaks", col=pal)
legend("topleft", legend = rownames(toplot), col=pal, pch=15, bty=F, pt.cex=2)
#plot.new()
par( las=1, mar = c(2,8,0,8))

for( n in 1:4){  
    barplot(as.matrix(genes_enhancer[n,]), ylim = c(0, max(genes_enhancer[n,]) + (max(genes_enhancer[n,]) /7 ) ) )
    mtext(rownames(genes_enhancer[n,]),at = 1, line=-0.5)
}

# ### Add stats prom before
# par(xpd = TRUE, las=2, mar = c(4,8,4,8))
# bp<-barplot(stat2[,3],  ylab ="# coaccessible sites (x1e3)",names.arg = cells,ylim=c(0,4e+2))
# text(bp, stat2[,3],labels=stat2[,3],pos=3)
# bp<-barplot(stat2[,1],  ylab ="# coaccessible\n enhancer-promoters(x1e3)",names.arg = cells, ylim=c(0,120))
# text(bp, stat2[,1],labels=stat2[,1],pos=3)
# bp<-barplot(stat2[,2],  ylab ="# coaccessible\n promoter-promoters(x1e3)",names.arg = cells,ylim=c(0,40))
# text(bp, stat2[,2],labels=stat2[,2],pos=3)
 dev.off()

png 
  2

In [29]:
ptomoter_qtls = read.table("summarized_caqtls/Promoter_associated_caQTLs.tsv",header=T)

In [30]:
sum(!duplicated(ptomoter_qtls$Feature))

[1] 3941

In [31]:
head(ptomoter_qtls)

,Feature,feature_gene,type,coacPeak,coaccess,dist,coac_gene,group
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
1,chr1:100259360-100260023,NA,EP,chr1:100816718-100819157,0.07022155,559134,CDC14A,b
2,chr1:108742333-108743769,SLC25A24,P,chr1:108319627-108320549,0.09679502,423220,NA,b
3,chr1:108742333-108743769,SLC25A24,PP,chr1:108505820-108508658,0.15152870,235111,VAV3,b
4,chr1:108742333-108743769,SLC25A24,PP,chr1:109203136-109204341,0.10216564,460572,HENMT1,b
5,chr1:108742333-108743769,SLC25A24,PP,chr1:109234521-109236324,0.10582430,492555,PRPF38B,b
6,chr1:1108854-1109596,TTLL10,P,chr1:1057321-1058169,0.08439527,51427,NA,b


In [32]:
ppp = subset(ptomoter_qtls, type!="EP")

In [33]:
sum(!duplicated(ppp$Feature))

[1] 1281

In [35]:
1281/6901

[1] 0.1856253

In [36]:
coc = read.table("run_cicero_paola/final_conns/annotated.coaccessible.matrix4s", header=T)

In [37]:
sum(!duplicated(coc$loops[coc$type=='PP']))

[1] 84788

In [38]:
sum(!duplicated(coc$loops[coc$type=='CP']))

[1] 254771